# Visualizations and figures for the paper

In [40]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs import Layout

## Voteview ideology scores

In [41]:
voteview = pd.read_csv("../data/voteview-HS117_members.csv", index_col="icpsr")

# Create a new column for marker colors based on 'party_code'
voteview["marker_color"] = voteview["party_code"].apply(
    lambda x: "red" if x == 100 else "blue" if x == 200 else "gray"
)

senate = voteview[voteview["chamber"] == "Senate"]
senate_democrats = senate[senate["party_code"] == 100]
senate_republicans = senate[senate["party_code"] == 200]
house = voteview[voteview["chamber"] == "House"]
house_democrats = house[house["party_code"] == 100]
house_republicans = house[house["party_code"] == 200]


marker_size = 10


fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    # vertical_spacing=0.9,
    subplot_titles=("Senate", "House"),
)

fig.add_trace(
    go.Scatter(
        x=senate_democrats["nominate_dim1"],
        mode="markers",
        marker_color=senate_democrats["marker_color"],
        name="Democrats",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=senate_republicans["nominate_dim1"],
        mode="markers",
        marker_color=senate_republicans["marker_color"],
        name="Republicans",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=house_democrats["nominate_dim1"],
        mode="markers",
        marker_color=house_democrats["marker_color"],
        name="Democrats",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=house_republicans["nominate_dim1"],
        mode="markers",
        marker_color=house_republicans["marker_color"],
        name="Republicans",
    ),
    row=2,
    col=1,
)

fig.update_yaxes(
    # showgrid=False,
    zerolinecolor="black",
    showticklabels=False,
)
fig.update_xaxes(range=[-1, 1])
fig.update_layout(
    showlegend=False,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    autosize=False,
    width=600,
    height=300,
    margin=dict(l=0, r=0, b=0, t=20, pad=4),
)
# save to png file
fig.write_image("../paper-figures/congress-nominate-dim1.png")
# fig.show()

## Donor ideology predictions in one dimension

In [42]:
# The ids of the politicians in the NVD dataset, stems from the littlesis ids
from utils import read_json
from typing import List

politician_info = read_json("../data/politicians_117.json")

In [43]:
data = {
    # Meta
    "marker_color": [],
    "nominate_dim1": [],
    "chamber": [],
    # Embedding from NVD TSNE
    "tsne-1": [],
    "tsne-2-0": [],
    "tsne-2-1": [],
    # Embedding from NVD PCA
    "pca-1": [],
    "pca-2-0": [],
    "pca-2-1": [],
    # Embedding from NVD ISOMAP
    "isomap-1": [],
    "isomap-2-0": [],
    "isomap-2-1": [],
}
for info in politician_info:
    try:
        if info["voteview_info"]["party_code"] == 100:
            marker_color = "red"
        elif info["voteview_info"]["party_code"] == 200:
            marker_color = "blue"
        else:
            marker_color = "gray"

        chamber: str = info["voteview_info"]["chamber"]
        nominate_dim1: float = info["voteview_info"]["nominate_dim1"]

        # Embedding from NVD TSNE
        TSNE_1: float = info["TSNE-1"][0]
        TSNE_2_0: float = info["TSNE-2"][0]
        TSNE_2_1: float = info["TSNE-2"][1]

        # Embedding from NVD PCA
        PCA_1: float = info["PCA-1"][0]
        PCA_2_0: float = info["PCA-2"][0]
        PCA_2_1: float = info["PCA-2"][1]

        # Embedding from NVD ISOMAP
        ISOMAP_1: float = info["Isomap-1"][0]
        ISOMAP_2_0: float = info["Isomap-2"][0]
        ISOMAP_2_1: float = info["Isomap-2"][1]

        # ADD VALUES
        data["marker_color"].append(marker_color)
        data["nominate_dim1"].append(nominate_dim1)
        data["chamber"].append(chamber)
        data["tsne-1"].append(TSNE_1)
        data["tsne-2-0"].append(TSNE_2_0)
        data["tsne-2-1"].append(TSNE_2_1)
        data["pca-1"].append(PCA_1)
        data["pca-2-0"].append(PCA_2_0)
        data["pca-2-1"].append(PCA_2_1)
        data["isomap-1"].append(ISOMAP_1)
        data["isomap-2-0"].append(ISOMAP_2_0)
        data["isomap-2-1"].append(ISOMAP_2_1)

    except KeyError as e:
        print(e)

'TSNE-1'
'TSNE-1'


In [44]:
assert len(data["tsne-1"]) == len(data["nominate_dim1"])
assert len(data["tsne-1"]) == len(data["chamber"])

In [45]:
df = pd.DataFrame(data)

senate = df[df["chamber"] == "Senate"]
senate_democrats = senate[senate["marker_color"] == "red"]
senate_republicans = senate[senate["marker_color"] == "blue"]
house = df[df["chamber"] == "House"]
house_democrats = house[house["marker_color"] == "red"]
house_republicans = house[house["marker_color"] == "blue"]

df.describe()

,nominate_dim1,tsne-1,tsne-2-0,tsne-2-1,pca-1,pca-2-0,pca-2-1,isomap-1,isomap-2-0,isomap-2-1
count,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000
mean,0.059377,0.032118,-0.039074,0.026857,-0.940958,-0.940958,-0.940958,-0.799452,-0.502083,-0.502083
std,0.461831,0.486606,0.509056,0.370347,0.119989,0.119989,0.017210,0.110801,0.092490,0.088280
min,-0.808000,-1.000000,-1.000000,-0.920588,-1.000000,-1.000000,-0.983926,-1.000000,-0.669489,-1.000000
25%,-0.378000,-0.358251,-0.451610,-0.253032,-0.992180,-0.992180,-0.948570,-0.832932,-0.530029,-0.499638
50%,-0.155000,0.158884,-0.133649,0.090150,-0.973066,-0.973066,-0.937469,-0.811417,-0.512070,-0.498925
75%,0.495000,0.399890,0.389590,0.292249,-0.937877,-0.937877,-0.930266,-0.795030,-0.498391,-0.494902
max,0.936000,1.000000,1.000000,0.841329,1.000000,1.000000,-0.700665,1.000000,1.000000,0.037958


In [50]:
marker_size = 10

for dimension_key in ["tsne-1", "pca-1", "isomap-1"]:
    print(df[dimension_key].min(), df[dimension_key].max())

    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        subplot_titles=(f"Senate-{dimension_key}", f"House-{dimension_key}"),
    )

    fig.add_trace(
        go.Scatter(
            x=senate_democrats[dimension_key],
            mode="markers",
            marker_color=senate_democrats["marker_color"],
            name="Democrats",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=senate_republicans[dimension_key],
            mode="markers",
            marker_color=senate_republicans["marker_color"],
            name="Republicans",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=house_democrats[dimension_key],
            mode="markers",
            marker_color=house_democrats["marker_color"],
            name="Democrats",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=house_republicans[dimension_key],
            mode="markers",
            marker_color=house_republicans["marker_color"],
            name="Republicans",
        ),
        row=2,
        col=1,
    )

    fig.update_yaxes(
        # showgrid=False,
        zerolinecolor="black",
        showticklabels=False,
    )
    fig.update_xaxes(range=[-1, 1])
    fig.update_layout(
        showlegend=False,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        autosize=False,
        width=600,
        height=300,
        margin=dict(l=0, r=0, b=0, t=20, pad=4),
    )

    if dimension_key == "tsne-1":
        fig.write_image(f"../paper-figures/congress-embedding-{dimension_key}.png")

    fig.show()

-1.0 1.0


-1.0 1.0


-1.0 1.0


## Donor ideology predictions in two dimensions

In [47]:
# df = pd.DataFrame(data)

# senate = df[df["chamber"] == "Senate"]
# senate_democrats = senate[senate["marker_color"] == "red"]
# senate_republicans = senate[senate["marker_color"] == "blue"]
# house = df[df["chamber"] == "House"]
# house_democrats = house[house["marker_color"] == "red"]
# house_republicans = house[house["marker_color"] == "blue"]


marker_size = 10

for dimension_key_0, dimension_key_1 in [
    ("tsne-2-0", "tsne-2-1"),
    ("pca-2-0", "pca-2-1"),
    ("isomap-2-0", "isomap-2-1"),
]:
    fig = make_subplots(
        rows=1,
        cols=2,
        shared_xaxes=True,
        # vertical_spacing=0.9,
        subplot_titles=(
            f"Senate-{dimension_key_0.split('-')[0]}",
            f"House-{dimension_key_0.split('-')[0]}",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=senate_democrats[dimension_key_0],
            y=senate_democrats[dimension_key_1],
            mode="markers",
            marker_color=senate_democrats["marker_color"],
            name="Democrats",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=senate_republicans[dimension_key_0],
            y=senate_republicans[dimension_key_1],
            mode="markers",
            marker_color=senate_republicans["marker_color"],
            name="Republicans",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=house_democrats[dimension_key_0],
            y=house_democrats[dimension_key_1],
            mode="markers",
            marker_color=house_democrats["marker_color"],
            name="Democrats",
        ),
        row=1,
        col=2,
    )

    fig.add_trace(
        go.Scatter(
            x=house_republicans[dimension_key_0],
            y=house_republicans[dimension_key_1],
            mode="markers",
            marker_color=house_republicans["marker_color"],
            name="Republicans",
        ),
        row=1,
        col=2,
    )

    fig.update_xaxes(range=[-1, 1])
    fig.update_layout(
        showlegend=False,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        autosize=False,
        width=600,
        height=300,
        margin=dict(l=0, r=0, b=0, t=20, pad=4),
    )

    if dimension_key_0 == "tsne-2-0":
        fig.write_image(
            f"../paper-figures/congress-embedding-{'-'.join(dimension_key_0.split('-')[:-1])}.png"
        )

    fig.show()
# save to png file
# fig.write_image("../paper-figures/congress-embedding_2.png")

## Summary statistics of donations

- Divided into republicans and democrates, what is the total amount of money donated to each party?
- How are the top 10 givers for each party?
- 

# Politician case study

Consider 4 four politicians. One from each extreme of the spectrum and two from the middle.

In [48]:
voteview = pd.read_csv("../data/voteview-HS117_members.csv", index_col="icpsr")
senate = voteview[voteview["chamber"] == "Senate"]
house = voteview[voteview["chamber"] == "House"]

# The two most extreme senators
senate.sort_values(by="nominate_dim1", inplace=True)
extremes = senate.iloc[[0, -1]]

# The most extreme republican and democrat
extreme_republican = extremes.loc[extremes.party_code == 200]
extreme_democrat = extremes.loc[extremes.party_code == 100]

# The most moderate republican and democrat
moderate_republican = (
    senate.loc[senate.party_code == 200]
    .sort_values(by="nominate_dim1", key=lambda x: abs(x))
    .iloc[[0]]
)

moderate_democrat = (
    senate.loc[senate.party_code == 100]
    .sort_values(by="nominate_dim1", key=lambda x: abs(x))
    .iloc[[0]]
)

for politician in [
    extreme_republican,
    extreme_democrat,
    moderate_republican,
    moderate_democrat,
]:
    party = "republican" if politician["party_code"].item() == 200 else "democrate"
    name = politician["bioname"].item()
    score = politician["nominate_dim1"].item()
    icpsr = politician.index.item()
    print(f"{party}\ticpsr:{icpsr}\tnominate_dim1 of {score}\t{name}")

republican	icpsr:42102	nominate_dim1 of 0.936	TUBERVILLE, Thomas Hawley (Tommy)
democrate	icpsr:41301	nominate_dim1 of -0.753	WARREN, Elizabeth
republican	icpsr:49703	nominate_dim1 of 0.117	COLLINS, Susan Margaret
democrate	icpsr:40915	nominate_dim1 of -0.058	MANCHIN, Joe, III


/var/folders/95/bytj9hyn2xl31dc1vxv5w9ww0000gn/T/ipykernel_52494/336948407.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Cosine similarity between the vectors

## Shortest paths between them

In [49]:
# Calculate the shortest path between all four senators
import pickle

G_largest = pickle.load(open("../data/littlesis-largest-component.pickle", "rb"))

# Calculate the shortest path between all four senators
from networkx import shortest_path

shortest_path(extreme_republican.index[0], extreme_democrat.index[0], G_largest)

TypeError: argument of type 'numpy.int64' is not iterable